DONE

In [ ]:
import sys
sys.path.append("..")

from main import *

from scipy.stats import pearsonr, spearmanr
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
from numba import njit
from scipy.stats import zscore

plt.rcParams['font.size'] = 18

if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu" 
    
print(device)

# Supplementary - Effect of network size

Contains analyses related to Supplementary Figure S4

In [ ]:
vertices = np.load('../Files/vertices_ellipse.npy').astype('float')
eigenmodes = np.load('../Files/eigenmodes_ellipse.npy')

ellipse = Geometry(vertices, eigenmodes)

In [ ]:
params = {'N_neurons': 10000,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

#### Averaging eigenmode-gradient correlations for varying network size $N$

In [ ]:
N_neurons = [10000, 5000, 2500, 1000]
N_runs = 10

mode_similarity_matrices = []

for N in N_neurons:

    params['N_neurons'] = N
    mode_similarity_matrices_ = []

    for _ in range(N_runs):
        simulator = Simulator(ellipse, ChaoticRNN, params)
        C = simulator.compute_average_correlations(n_iters=100, T=500)
        mode_similarity, _ = simulator.compute_geometric_mapping(C, N_modes=50)
        mode_similarity_matrices_.append(mode_similarity)

    mode_similarity_matrices.append(mode_similarity_matrices_)

In [ ]:
save = False
if save:
    np.save('../Results/supp_network_size_matrices.npy', mode_similarity_matrices)

#### Analyzing results

In [ ]:
mode_similarity_matrices = np.load('../Results/supp_network_size_matrices.npy')

In [ ]:
scores = np.zeros((mode_similarity_matrices.shape[0], mode_similarity_matrices.shape[1]))

for i in range(scores.shape[0]):
    for j in range(scores.shape[1]):
        scores[i, j] = np.mean(np.abs(np.diag(mode_similarity_matrices[i, j])))

Plotting average eigenmode-gradient correlations for multiple simulations and varying $N$

In [ ]:
for i in range(scores.shape[0]):
    plt.scatter([4 - i] * scores.shape[1], scores[i], color='black')

In [ ]:
avg_matrices = np.mean(np.abs(mode_similarity_matrices), axis=1)

In [ ]:
for m in avg_matrices:

    plt.imshow(m, cmap='Reds', vmin=0, vmax=1.0)
    plt.show()

# Saving images of ellipses

Rendering .png images of 3D scatter plots of the ellipsoid for the figure later.

In [ ]:
coords_list, gradients_list = [], []

In [ ]:
params = {'N_neurons': 2500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

#### 1000 neurons

In [ ]:
params['N_neurons'] = 1000

simulator = Simulator(ellipse, ChaoticRNN, params)
C = simulator.compute_average_correlations(n_iters=100, T=500)
_, _, gradients = simulator.compute_geometric_mapping(C, N_modes=50, return_gradients=True)

In [ ]:
coords_list.append(np.copy(simulator.coordinates))
gradients_list.append(np.copy(gradients[0]))

#### 2500 neurons

In [ ]:
params['N_neurons'] = 2500

simulator = Simulator(ellipse, ChaoticRNN, params)
C = simulator.compute_average_correlations(n_iters=100, T=500)
_, _, gradients = simulator.compute_geometric_mapping(C, N_modes=50, return_gradients=True)

In [ ]:
coords_list.append(np.copy(simulator.coordinates))
gradients_list.append(np.copy(gradients[0]))

#### 5000 neurons

In [ ]:
params['N_neurons'] = 5000

simulator = Simulator(ellipse, ChaoticRNN, params)
C = simulator.compute_average_correlations(n_iters=100, T=500)
_, _, gradients = simulator.compute_geometric_mapping(C, N_modes=50, return_gradients=True)

In [ ]:
coords_list.append(np.copy(simulator.coordinates))
gradients_list.append(np.copy(gradients[0]))

#### 10000 neurons

In [ ]:
params['N_neurons'] = 10000

simulator = Simulator(ellipse, ChaoticRNN, params)
C = simulator.compute_average_correlations(n_iters=100, T=500)
_, _, gradients = simulator.compute_geometric_mapping(C, N_modes=50, return_gradients=True)

In [ ]:
coords_list.append(np.copy(simulator.coordinates))
gradients_list.append(np.copy(gradients[0]))

#### Rendering figures

In [ ]:
figures_list = []
signs = [1, 1, -1, -1]

for i in range(len(coords_list)):

    coords = coords_list[i]
    gradient = gradients_list[i]

    fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(5, 5), dpi=300)
    ax.scatter(coords[:, 0] + 0.5, coords[:, 1] + 0.5, coords[:, 2] + 0.5, c=signs[i] * gradient, alpha=0.5, cmap='coolwarm', edgecolor='None', s=25)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
    ax.set_axis_off()
    plt.tight_layout(pad=0)
    figures_list.append(figure_to_array(fig))
    plt.show()

In [ ]:
np.save('Results/supp_network_size_figures_ellipse.npy', figures_list)

# Rendering supplementary figure

In [ ]:
from visualization import *

def zoom_crop(array, factor=2, x_offset=0, y_offset=0):
    if factor != 1:
        L = array.shape[0]
        delta = int((1 - (1 / factor)) * L / 2)
        return array[delta+y_offset:-delta+y_offset, delta+x_offset:-delta+x_offset, :]
    else:
        return array

In [ ]:
mode_similarity_matrices = np.load('../Results/supp_network_size_matrices.npy')

matrices = []
for i in range(mode_similarity_matrices.shape[0]):
    matrices.append(np.mean(np.abs(mode_similarity_matrices[i]), axis=0))

scores = np.zeros((mode_similarity_matrices.shape[0], mode_similarity_matrices.shape[1]))
for i in range(scores.shape[0]):
    for j in range(scores.shape[1]):
        scores[i, j] = np.mean(np.abs(np.diag(mode_similarity_matrices[i, j])))
scores = np.flip(scores, axis=0)  

figures = np.load('../Results/supp_network_size_figures_ellipse.npy')

In [ ]:
fig = PaperFigure(figsize=(7, 3))

fig.set_tick_length(2)
fig.set_font_size(6)
fig.add_background()

w = 0

# Setting up axes ----------------------------------------------------------------------------------

fig.add_axes('boxplot', (0, 0), 1, 1.75)

w = 1.1
x = 1.5
pad = ((7 - x) - 4 * w) / 3

for i in range(4):
    fig.add_axes(f'matrix{i}', (x + i * (w + pad), 0), w, w)

for i in range(4):
    fig.add_axes(f'ellipse{i}', (x + i * (w + pad), w + pad/2), w, (4/3) * w)

fig.set_line_thickness(0.5)

# Filling axes -------------------------------------------------------------------------------------

for i in range(4):
    ax = fig.axes[f'matrix{i}']
    ax.imshow(matrices[3-i], cmap='Reds', vmin=0, vmax=1)
    ax.set_xticks([])
    ax.set_yticks([])
    
center = (760, 735)
dx, dy = 300, 400
for i in range(4):
    ax = fig.axes[f'ellipse{i}']
    ax.imshow(figures[i][center[1]-dy:center[1]+dy, center[0]-dx:center[0]+dx])
    ax.axis('off')

colors = color_gradient([0, 0, 0], plt.get_cmap('Reds')(200)[:3], 4)
ax = fig.axes['boxplot']
for i in range(4):
    ax.scatter(np.random.uniform(i-0.1, i+0.1, len(scores[i])), scores[i], color=colors[i], edgecolor='None', s=5, alpha=0.5)
    bp = ax.boxplot(scores[i], positions=[i], patch_artist=True, widths=0.4, whis=[5, 95])
    set_boxplot_color(bp, colors[i], marker='None', linewidth=0.75)
ax.set_xlim([-0.5, 4])
ax.set_ylim([0.4, 0.7])
ax.set_xticks([])
ax.spines[['top', 'right']].set_visible(False)

fig.show()

In [ ]:
fig.save('../Figures/supp_network_size_incomplete.svg')

#### Statistics

In [ ]:
scores_list = []
for i in range(scores.shape[0]):
    scores_list.append(scores[i])

In [ ]:
groups_ANOVA_Tukey(scores_list)